# Calculation of volume constraints for Si-composite anodes

**Authors:**
- Carl Erik L. Foss, IFE
- Jan Petter Maehlen, IFE

**Last update:** 2021.04.23

**Acknowledgements:** Elkem and Research Council of Norway

## Imports and setup

In [ ]:
# from IPython.display import display, Math, Latex
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Information

(or maybe only a link to information)

Fig 1. Expansion

<img src="SiliconExpansion_fig_001.png">

Fig 2. Initial vs. expanded

<img src="SiliconExpanded_fig_001.png">

## DEV: Objective of notebook

1. Calculate porosity based on measured mass, thickness and either area or diamter of electrode.
2. Calculate change in porosity based on lithiation level
3. Should be possible to generate a xy plot of porosity vs lithiation level for any given Silicon/Graphite/Other ratio
4. Should be possible to generate a plot showing volumetric and gravimetric capacity as a function of Silicon/Graphite/Other ratio and lithiation level

In the end, the functions can be moved out of the notebook into a python module (for example a .py file in the same directory)

### Next (other notebook?)
1. Interactive notebook
2. Calculate energy density of "real" cell given material prms for anode and cathode



## Input

Used for development

In [ ]:
# Needed input prms (constants)

# expansion coeffs
expansion_coeff_si       = 2.8 
expansion_coeff_graphite = 0.1
expansion_coeff_dead     = 0.0

# densities
rho_si = 2.329           # g/ccm
rho_graphite = 2.1       # g/ccm
rho_dead = 1.8           # g/ccm

# max capacities
maxcap_si = 3579.0       # mAh/g
maxcap_graphite = 372.0  # mAh/g
maxcap_dead = 0.0        # mAh/g


In [ ]:
# Needed input prms (variables)

# composition
mass_fraction_silicon  = 0.6
mass_fraction_graphite = 0.3
mass_fraction_dead     = 0.1

# electrode / cell
radius = 0.5*1.5

eps = 1.0  # allowed expansion coef for electrode (1.0 means no room available)

# cut-off capacities
max_cutoff_cap_si = 3579.0       # mAh/g
max_cutoff_cap_graphite = 372.0  # mAh/g
max_cutoff_cap_dead = 0.0        # mAh/g

# porosity range
porosity = 0.5 # initial porosity
porosity_limit = 0.0 # final limit for porosity

## Utility functions

In [ ]:
def density_composite(densities, mass_fractions, porosity = 0.0, debug=False):
    """Calculate density of composite.
    
    Parameters
    ----------
    densities : list of floats
        The densities of the composite components (in g/ccm).
    mass_fractions : list of floats
        The mass fraction of each composite component (in g).
    porosity : float 
        Porosity (from 0.0 to 1.0, where 0.0 means no porosity).
    debug : bool
        Print measeages to stdout if True.
        
    Returns
    -------
    Float
        The calculated density
    """
    
    
    total_mass = np.sum(mass_fractions)
    
    total_volume = 0.0
    for d,m in zip(densities, mass_fractions):
        if debug:
            print(f"density: {d}\nfraction: {m}")
        total_volume += m/d
    try:
        total_volume = total_volume/(1-porosity)
    except ZeroDivisionError:
        print("ERROR! You have a porosity of 1.0 (meaning your electrode contains nothing)")
        return None
    
    density = total_mass/total_volume
    if debug:
        print(f"Density: {density}")
    return density

In [ ]:
def real_density_composite_old(densities, mass_fractions, normalize=True):
    """Calculate density of composite.
    
    Parameters
    ----------
    densities : list of floats
        The densities of the composite components (in g/ccm).
    mass_fractions : list of floats
        The mass fraction of each composite component (in g).
    normalize : bool
        Normalize the mass fraction so that they sum to one.
        
    Returns
    -------
    Float
        The calculated density
        
    Notes
    -----
    This function can be used if you assume the internal porosity
        is zero. This function uses numpy arrays and is faster
        thant the density_composite function
    
    """
    
    mass_fractions = np.array(mass_fractions)
    densities = np.array(densities)
    
    if normalize:
        mass_fractions = mass_fractions / np.sum(mass_fractions)
    
    density = np.sum(mass_fractions * densities)
    return density

In [ ]:
# test density_composite
densities = [rho_si, rho_graphite, rho_dead]
mfracs = [mass_fraction_silicon, mass_fraction_graphite, mass_fraction_dead]
density = density_all(densities, mfracs, porosity = 0.0)

print(f"The calculated density is {density:0.2f} g/ccm")

# test real_density_composite
rdensity = real_density_composite(densities, mfracs)

print(f"The calculated (real) density is {rdensity:0.2f} g/ccm")

if abs(rdensity-density) != 0.0:
    print("NOT EQUAL. WHY? OH-GOD WHYYYYY?")

## 1. Calculate porosity based on measured mass, thickness and area

In [ ]:
mass_of_electrode = 2.23/1000  # g
thickness_of_electrode = 12/10000 # cm
area_of_electrode = 0.25 * np.pi * 1.5**2 # cm2 

In [ ]:
def porosity_theoretical(mass, thickness, area, density):
    volume_real = thickness * area
    volume_calc = mass / density
    porosity = 1 - (volume_calc/volume_real)
    
    return porosity

In [ ]:
density = density_composite(densities, mfracs, porosity = 0.0)
porosity = porosity_theoretical(mass_of_electrode, thickness_of_electrode, area_of_electrode, density)

In [ ]:
print(density)
print(porosity)

## 2. Calculate change in porosity based on lithiation level

### 2.1 density

In [ ]:
def updated_density_single(x, old_density, expansion_coeff):
    # Assuming linear expansion and lithiation degree up to x (x=1 is max).
    # Expansion_coeff (expansion of delta V) is defined so that 0 will give a non-changing density.
    #    For fully lithiated compound, the volume will change from V  V + dV, where dV = expansion_coeff * V
    d = old_density / (1 + expansion_coeff * x)
    return d


In [ ]:
x = np.linspace(0.0, 1.0, 100)
d = updated_density_single(x, rho_si, expansion_coeff_si)


In [ ]:
print(f"Evolution of density for Silicon as a function of lithiation degree.")
print(f" - Initial density: {rho_si} g/ccm")
print(f" - Expansion coeff: {expansion_coeff_si}")
plt.plot(x,d)
plt.ylim(0,3)
plt.ylabel("density SiLix, rho")
plt.xlabel("lith. degree, x");

In [ ]:
x = np.linspace(0.0, 1.0, 100)
mass_fracs = [0.4, 0.4, 0.2]


d_Si = updated_density_single(x, rho_si, expansion_coeff_si)
d_G = updated_density_single(x, rho_graphite, expansion_coeff_graphite)
d_D = updated_density_single(x, rho_dead, expansion_coeff_dead)


d_comp_v = np.array([d_Si, d_G, d_D]).T  # shape: (100, 3)
mfracs_v = np.ones((100, 3)) * mass_fracs / np.sum(mass_fracs) # shape: (100, 3)

d = updated_density_single(x, rho_si, expansion_coeff_si)

dd = np.sum(d_comp_v * mfracs_v, axis=1)

In [ ]:
print(f"Evolution of density for composite as a function of lithiation degree.")
plt.plot(x,d, label="pure Si")
plt.plot(x,dd, label = "composite")
plt.legend()
plt.ylim(1,2.5)
plt.xlim(0, 1)
plt.ylabel("density SiLix, rho")
plt.xlabel("lith. degree, x");

### 2.2 porosity